# Probleme de clasificare a textelor

## Obiective
* extragerea de caracteristici din informatiile textuale

## Cuvinte cheie:
* date de antrenare si date de testare 
* atribute/catacteristici ale datelor
* etichete ale datelor
* normalizare date 
* model de clasificare
* acuratetea clasificarii

## Exemple

### Demo1

#### Problema: Ce fel de mesaje primesti in Inbox? 
Se doreste clasificarea unor mesaje in doua categorii (spam si ham). Pentru fiecare mesaj se cunoaste textul aferent lui si categoria din care face parte. Să se rezolve problema, implementându-se rutine pentru clasificarea mesajelor.

Proces:
* Se pleaca de la un set de date format din textul mesajelor precum cel din fisierul spam csv [link](data/spam.csv).
* Se imparte setul de date in date de antrenament si in date de test
* Se extrag anumite caracteristici din textul mesajelor folosind diferite reprezentari precum:
    - Bag of Words
    - TF-IDF
    - Word2Vec
* invatare model 
* calcul metrici de performanta

In [1]:
import csv
import os

#### Pasul 1 - incarcare date

In [2]:
# load some data
crtDir =  os.getcwd()
fileName = os.path.join(crtDir, 'data', 'spam.csv')

data = []
with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            dataNames = row
        else:
            data.append(row)
        line_count += 1

inputs = [data[i][0] for i in range(len(data))][:100]
outputs = [data[i][1] for i in range(len(data))][:100]
labelNames = list(set(outputs))

print(inputs[:2])
print(labelNames[:2])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...']
['spam', 'ham']


#### Pasul 2 - impartire date (antrenament si test)

In [3]:
# prepare data for training and testing

import numpy as np

np.random.seed(5)
# noSamples = inputs.shape[0]
noSamples = len(inputs)
indexes = [i for i in range(noSamples)]
trainSample = np.random.choice(indexes, int(0.8 * noSamples), replace = False)
testSample = [i for i in indexes  if not i in trainSample]

trainInputs = [inputs[i] for i in trainSample]
trainOutputs = [outputs[i] for i in trainSample]
testInputs = [inputs[i] for i in testSample]
testOutputs = [outputs[i] for i in testSample]

print(trainInputs[:3])

['Today is \\song dedicated day..\\" Which song will u dedicate for me? Send this to all ur valuable frnds but first rply me..."', 'K tell me anything about you.', "Didn't you get hep b immunisation in nigeria."]


#### Pasul 3 - extragere caracteristici

In [ ]:
# extract some features from the raw text

# # representation 1: Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

trainFeatures = vectorizer.fit_transform(trainInputs)
testFeatures = vectorizer.transform(testInputs)

# vocabulary size
print("vocab size: ", len(vectorizer.vocabulary_),  " words")
# no of emails (Samples)
print("traindata size: ", len(trainInputs), " emails")
# shape of feature matrix
print("trainFeatures shape: ", trainFeatures.shape)

# vocabbulary from the train data 
print('some words of the vocab: ', vectorizer.get_feature_names_out()[-20:])
# extracted features
print('some features: ', trainFeatures.toarray()[:3])




vocab size:  604  words
traindata size:  80  emails
trainFeatures shape:  (80, 604)
some words of the vocab:  ['world' 'worried' 'wun' 'www' 'xuhui' 'xx' 'xxx' 'yeah' 'year' 'yes'
 'yesterday' 'yo' 'you' 'your' 'yours' 'yourself' 'yummy' 'yup' 'ì_' 'ì¼1']
some features:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [4]:
# representation 2: tf-idf features - word granularity
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=50)

trainFeatures = vectorizer.fit_transform(trainInputs)
testFeatures = vectorizer.transform(testInputs)

# vocabbulary from the train data 
print('vocab: ', vectorizer.get_feature_names_out()[:10])
# extracted features
print('features: ', trainFeatures.toarray()[:3])


vocab:  ['all' 'already' 'and' 'anything' 'at' 'be' 'but' 'call' 'can' 'did']
features:  [[0.28313142 0.         0.         0.         0.         0.
  0.31579324 0.         0.         0.         0.         0.24009176
  0.         0.         0.         0.         0.         0.
  0.         0.         0.26643568 0.         0.         0.
  0.58555921 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.32990928 0.17661816 0.         0.
  0.32990928 0.         0.         0.31579324 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.70343249 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.59419418 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0. 

In [ ]:
!pip install gensim



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# representation 3: embedded features extracted by a pre-train model (in fact, word2vec pretrained model)

import gensim 

# Load Google's pre-trained Word2Vec 
crtDir =  os.getcwd()
modelPath = os.path.join(crtDir, 'models', 'GoogleNews-vectors-negative300.bin')

word2vecModel300 = gensim.models.KeyedVectors.load_word2vec_format(modelPath, binary=True) 
print(word2vecModel300.most_similar('support'))
print("vec for house: ", word2vecModel300["house"])

[('supporting', 0.6251285076141357), ('suport', 0.6071150302886963), ('suppport', 0.6053199768066406), ('Support', 0.6044272780418396), ('supported', 0.6009396314620972), ('backing', 0.6007589101791382), ('supports', 0.5269277691841125), ('assistance', 0.5207138061523438), ('sup_port', 0.5192490220069885), ('supportive', 0.5110024809837341)]
vec for house:  [ 1.57226562e-01 -7.08007812e-02  5.39550781e-02 -1.89208984e-02
  9.17968750e-02  2.55126953e-02  7.37304688e-02 -5.68847656e-02
  1.79687500e-01  9.27734375e-02  9.03320312e-02 -4.12109375e-01
 -8.30078125e-02 -1.45507812e-01 -2.37304688e-01 -3.68652344e-02
  8.74023438e-02 -2.77099609e-02  1.13677979e-03  8.30078125e-02
  3.57421875e-01 -2.61718750e-01  7.47070312e-02 -8.10546875e-02
 -2.35595703e-02 -1.61132812e-01 -4.78515625e-02  1.85546875e-01
 -3.97949219e-02 -1.58203125e-01 -4.37011719e-02 -1.11328125e-01
 -1.05957031e-01  9.86328125e-02 -8.34960938e-02 -1.27929688e-01
 -1.39648438e-01 -1.86523438e-01 -5.71289062e-02 -1.176

In [ ]:
word = "casuta"
if (word in word2vecModel300.index_to_key):
    print("vec for house: ", word2vecModel300[word])
else:
    print("word was not found!")

word was not found!


In [ ]:
def featureComputation(model, data):
    features = []
    phrases = [ phrase.split() for phrase in data]
    for phrase in phrases:
        # compute the embeddings of all the words from a phrase (words of more than 2 characters) known by the model
        vectors = [model[word] for word in phrase if (len(word) > 2) and (word in model.index_to_key)]
        if len(vectors) == 0:
            result = [0.0] * model.vector_size
        else:
            result = np.sum(vectors, axis=0) / len(vectors)
        features.append(result)
    return features

trainFeatures = featureComputation(word2vecModel300, trainInputs)
testFeatures = featureComputation(word2vecModel300, testInputs)



#### Pasul 4 - antrenare model de invatare supervizata

In [7]:
# supervised classification of data

from sklearn.linear_model import LogisticRegression

supervisedClassifier = LogisticRegression(max_iter=1000)

supervisedClassifier.fit(trainFeatures, trainOutputs)

LogisticRegression(max_iter=1000)

#### Pasul 5 - testare model

In [13]:
computedTestOutputs = supervisedClassifier.predict(testFeatures)
print(computedTestIndexes)
# computedTestOutputs = [labelNames[value] for value in computedTestIndexes]
for i in range(0, len(testInputs)):
    print(testInputs[i], " -> ", computedTestOutputs[i])

['ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham']
As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune  ->  ham
WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.  ->  ham
XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL  ->  ham
Oh k...i'm watching here:)  ->  ham
Did you catch the bus ? Are you frying an egg ? Did you make a tea? Are you eating your mom's left over dinner ? Do you feel my Love ?  ->  ham
Wait that's still not all that clear, were you not sure about me being sarcastic or that that's why x doesn't want to live with us  ->  ham
Great! I hope you like your man well endowed. I am  &lt;#&gt

#### Pasul 6 - calcul metrici de performanta

In [15]:
from sklearn.metrics import accuracy_score

# just supposing that we have the true labels
print("acc: ", accuracy_score(testOutputs, computedTestOutputs))


acc:  0.85
